In [1]:
!pip install torch_geometric

In [2]:
!pip install torch-scatter==latest+cu101 torch-sparse==latest+cu101 -f https://s3.eu-central-1.amazonaws.com/pytorch-geometric.com/whl/torch-1.4.0.html

Looking in links: https://s3.eu-central-1.amazonaws.com/pytorch-geometric.com/whl/torch-1.4.0.html
  Using cached https://s3.eu-central-1.amazonaws.com/pytorch-geometric.com/whl/torch-1.4.0/torch_scatter-latest%2Bcu101-cp36-cp36m-linux_x86_64.whl
  Using cached https://s3.eu-central-1.amazonaws.com/pytorch-geometric.com/whl/torch-1.4.0/torch_sparse-latest%2Bcu101-cp36-cp36m-linux_x86_64.whl
  Found existing installation: torch-scatter 2.0.4
    Uninstalling torch-scatter-2.0.4:
      Successfully uninstalled torch-scatter-2.0.4
  Found existing installation: torch-sparse 0.6.0
    Uninstalling torch-sparse-0.6.0:
      Successfully uninstalled torch-sparse-0.6.0


In [0]:
import sys
sys.path.insert(0, '/content/drive/My Drive')

In [0]:
import torch
import numpy as np
import networkx as nx
import torch.nn as nn
import time
import torch.nn.functional as F

In [0]:
from utils import EarlyStopping

In [0]:
from torch_geometric.datasets import Planetoid

In [0]:
Cora = Planetoid(root='/content/drive/My Drive/', name='Cora')
cora = Cora[0]

In [0]:
C_S = Planetoid(root='/content/drive/My Drive/', name='CiteSeer')
cite_seer = C_S[0]

In [0]:
PM = Planetoid(root='/content/drive/My Drive/', name='PubMed')
pub_med = PM[0]

In [0]:
data = pub_med
num_features = data.num_features
num_classes = len(set(data.y.numpy()))

In [0]:
from torch_geometric.nn import GCNConv, AGNNConv, SAGEConv,GATConv

In [0]:
cora = torch.load('/content/drive/My Drive/Connected_graph/Con_Cora.pt')

In [0]:
cite_seer = torch.load('/content/drive/My Drive/Connected_graph/Con_Cite.pt')


In [0]:
def Gen_data(dataset, num_features,seed):
    np.random.seed(seed)
    data = dataset.clone()
    rx = np.random.normal(0, 1, size=(data.num_nodes, num_features) )
    #rx = np.random.random((data.num_nodes, num_features))
    rx = torch.FloatTensor(rx)
    data.x = rx
    return data

In [0]:
# Cora
cora_seed = 1024
N_F = 300
r_cora = Gen_data(cora, N_F, cora_seed)

In [0]:
#Cite_Seer
cite_seer_seed = 1024
N_F = 500
r_citeseer = Gen_data(cite_seer, N_F, cite_seer_seed)

In [0]:
#pubmed
pub_med_seed = 1024
N_F = 1000
r_pubmed = Gen_data(pub_med, N_F, pub_med_seed)

In [0]:
data = r_cora
num_features = data.num_features
num_classes = len(set(data.y.numpy()))

In [0]:
r_cora.x

tensor([[ 2.1244,  0.2526,  1.4542,  ..., -0.1193,  0.8857, -0.4781],
        [-0.0751,  2.1528, -0.0590,  ..., -1.2324, -0.7668,  1.6055],
        [-1.7063, -1.0334,  1.3078,  ...,  0.8975,  0.1197,  0.2837],
        ...,
        [ 0.9521,  0.6183, -0.4583,  ...,  1.3327,  0.1096,  0.5242],
        [ 1.0178, -0.6566,  0.2029,  ..., -0.4533, -0.8218,  0.7424],
        [-0.8163, -1.1790,  0.1091,  ...,  0.1708,  0.2179, -1.2352]])

# GCN

In [0]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(num_features, 64, cached=True)
        self.conv2 = GCNConv(64, num_classes, cached=True)

    def forward(self):
        x, edge_index, edge_weight = data.x, data.edge_index, data.edge_attr
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)
        return F.log_softmax(x, dim=1)

In [19]:
gcns = []
for _ in range(10):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model, data = Net().to(device), data.to(device)
  '''optimizer = torch.optim.Adam([
      dict(params=model.reg_params, weight_decay=5e-4),
      dict(params=model.non_reg_params, weight_decay=0)
  ], lr=0.01)'''
  #r_cora_gcn
  optimizer = torch.optim.Adam(model.parameters(), lr=0.02, weight_decay=5e-4)
  #r_pubmed_gcn
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.002, weight_decay=5e-4)#0.395
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.5, weight_decay=5e-1)#0.424
  #r_cite_seer_gcn
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.02, weight_decay=5e-4)

  def train():
      model.train()
      optimizer.zero_grad()
      loss = F.nll_loss(model()[data.train_mask], data.y[data.train_mask])
      loss.backward()
      optimizer.step()
      return loss

  def test(mask):
      model.eval()
      with torch.no_grad():
          logits = model()
          pred = logits[mask].max(1)[1]
          acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
          
      return acc


  early_stop = True
  if early_stop:
      stopper = EarlyStopping(patience=100)
  dur = []
  print(model)

  for epoch in range(1, 101):
      model.train()
      if epoch >= 3:
          t0 = time.time()
      loss = train()
      
      if epoch >= 3:
          dur.append(time.time() - t0)
      
      val_acc = test(data.val_mask)
      
      if early_stop:
          if stopper.step(val_acc, model):   
              break
      
      '''print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | "
                " ValAcc {:.4f}".
                format(epoch, np.mean(dur), loss, val_acc))'''
      
  print() # 空出一行
      
  if early_stop:
      model.load_state_dict(torch.load('es_checkpoint.pt'))
  test_acc = test(data.test_mask)
  gcns.append(test_acc)
  print("Test Accuracy {:.4f}".format(test_acc))

Net(
  (conv1): GCNConv(300, 64)
  (conv2): GCNConv(64, 7)
)

Test Accuracy 0.5038
Net(
  (conv1): GCNConv(300, 64)
  (conv2): GCNConv(64, 7)
)

Test Accuracy 0.5049
Net(
  (conv1): GCNConv(300, 64)
  (conv2): GCNConv(64, 7)
)

Test Accuracy 0.4973
Net(
  (conv1): GCNConv(300, 64)
  (conv2): GCNConv(64, 7)
)

Test Accuracy 0.5071
Net(
  (conv1): GCNConv(300, 64)
  (conv2): GCNConv(64, 7)
)

Test Accuracy 0.4951
Net(
  (conv1): GCNConv(300, 64)
  (conv2): GCNConv(64, 7)
)

Test Accuracy 0.4809
Net(
  (conv1): GCNConv(300, 64)
  (conv2): GCNConv(64, 7)
)

Test Accuracy 0.5038
Net(
  (conv1): GCNConv(300, 64)
  (conv2): GCNConv(64, 7)
)

Test Accuracy 0.5027
Net(
  (conv1): GCNConv(300, 64)
  (conv2): GCNConv(64, 7)
)

Test Accuracy 0.4732
Net(
  (conv1): GCNConv(300, 64)
  (conv2): GCNConv(64, 7)
)

Test Accuracy 0.4940


In [20]:
np.round(gcns,3).tolist()

[0.504, 0.505, 0.497, 0.507, 0.495, 0.481, 0.504, 0.503, 0.473, 0.494]

In [21]:
np.mean(gcns)

0.4962841530054646

# SAGE

In [0]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = SAGEConv(num_features, 64, normalize=False)
        self.conv2 = SAGEConv(64, num_classes, normalize=False)

    def forward(self):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.5, training=self.training)
        #x = F.dropout(x, training=self.training)
        
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

In [0]:
data = r_citeseer
num_features = data.num_features
num_classes = len(set(data.y.numpy()))

In [25]:
sages = []
for _ in range(10):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model, data = Net().to(device), data.to(device)
  '''optimizer = torch.optim.Adam([
      dict(params=model.reg_params, weight_decay=5e-4),
      dict(params=model.non_reg_params, weight_decay=0)
  ], lr=0.01)'''
  #r_cora_sage
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.02, weight_decay=5e-4)
  #pubmed
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.5, weight_decay=5e-1)
  #r_cite_seer_sage
  optimizer = torch.optim.Adam(model.parameters(), lr=0.02, weight_decay=5e-4)

  def train():
      model.train()
      optimizer.zero_grad()
      loss = F.nll_loss(model()[data.train_mask], data.y[data.train_mask])
      loss.backward()
      optimizer.step()
      return loss

  def test(mask):
      model.eval()
      with torch.no_grad():
          logits = model()
          pred = logits[mask].max(1)[1]
          acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
          
      return acc


  early_stop = True
  if early_stop:
      stopper = EarlyStopping(patience=100)
  dur = []
  print(model)

  for epoch in range(1, 101):
      model.train()
      if epoch >= 3:
          t0 = time.time()
      loss = train()
      
      if epoch >= 3:
          dur.append(time.time() - t0)
      
      val_acc = test(data.val_mask)
      
      if early_stop:
          if stopper.step(val_acc, model):   
              break
      
      '''print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | "
                " ValAcc {:.4f}".
                format(epoch, np.mean(dur), loss, val_acc))'''
      
  print() # 空出一行
      
  if early_stop:
      model.load_state_dict(torch.load('es_checkpoint.pt'))
  test_acc = test(data.test_mask)
  sages.append(test_acc)
  print("Test Accuracy {:.4f}".format(test_acc))

Net(
  (conv1): SAGEConv(500, 64)
  (conv2): SAGEConv(64, 6)
)

Test Accuracy 0.4344
Net(
  (conv1): SAGEConv(500, 64)
  (conv2): SAGEConv(64, 6)
)

Test Accuracy 0.3997
Net(
  (conv1): SAGEConv(500, 64)
  (conv2): SAGEConv(64, 6)
)

Test Accuracy 0.3997
Net(
  (conv1): SAGEConv(500, 64)
  (conv2): SAGEConv(64, 6)
)

Test Accuracy 0.3982
Net(
  (conv1): SAGEConv(500, 64)
  (conv2): SAGEConv(64, 6)
)

Test Accuracy 0.4087
Net(
  (conv1): SAGEConv(500, 64)
  (conv2): SAGEConv(64, 6)
)

Test Accuracy 0.4133
Net(
  (conv1): SAGEConv(500, 64)
  (conv2): SAGEConv(64, 6)
)

Test Accuracy 0.4057
Net(
  (conv1): SAGEConv(500, 64)
  (conv2): SAGEConv(64, 6)
)

Test Accuracy 0.4072
Net(
  (conv1): SAGEConv(500, 64)
  (conv2): SAGEConv(64, 6)
)

Test Accuracy 0.3937
Net(
  (conv1): SAGEConv(500, 64)
  (conv2): SAGEConv(64, 6)
)

Test Accuracy 0.4103


In [26]:
np.round(sages,3).tolist()

[0.434, 0.4, 0.4, 0.398, 0.409, 0.413, 0.406, 0.407, 0.394, 0.41]

In [27]:
np.mean(sages)

0.4070889894419306

# AGNN

In [0]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.lin1 = torch.nn.Linear(num_features, 64)
        self.prop1 = AGNNConv(requires_grad=True)
        self.prop2 = AGNNConv(requires_grad=True)
        self.lin2 = torch.nn.Linear(64, num_classes)

    def forward(self):
        x = F.dropout(data.x, training=self.training)
        x = F.relu(self.lin1(x))
        x = self.prop1(x, data.edge_index)
        x = self.prop2(x, data.edge_index)
        x = F.dropout(x, training=self.training)
        x = self.lin2(x)
        return F.log_softmax(x, dim=1)

In [0]:
data = r_pubmed
num_features = data.num_features
num_classes = len(set(data.y.numpy()))

In [0]:
agnns = []
for _ in range(10):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model, data = Net().to(device), data.to(device)

  #r_cora
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.02, weight_decay=5e-4)
  #r-pubmed
  optimizer = torch.optim.Adam(model.parameters(), lr=0.5, weight_decay=5e-1)
  #r-cite_seer
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.5, weight_decay=5e-1)
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.02, weight_decay=5e-4)

  def train():
      model.train()
      optimizer.zero_grad()
      loss = F.nll_loss(model()[data.train_mask], data.y[data.train_mask])
      loss.backward()
      optimizer.step()
      return loss

  def test(mask):
      model.eval()
      with torch.no_grad():
          logits = model()
          pred = logits[mask].max(1)[1]
          acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
          
      return acc


  early_stop = True
  if early_stop:
      stopper = EarlyStopping(patience=100)
  dur = []
  print(model)

  for epoch in range(1, 101):
      model.train()
      if epoch >= 3:
          t0 = time.time()
      loss = train()
      
      if epoch >= 3:
          dur.append(time.time() - t0)
      
      val_acc = test(data.val_mask)
      
      if early_stop:
          if stopper.step(val_acc, model):   
              break
      
      '''print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | "
                " ValAcc {:.4f}".
                format(epoch, np.mean(dur), loss, val_acc))'''
      
  print() 
      
  if early_stop:
      model.load_state_dict(torch.load('es_checkpoint.pt'))
  test_acc = test(data.test_mask)
  agnns.append(test_acc)
  print("Test Accuracy {:.4f}".format(test_acc))

Net(
  (lin1): Linear(in_features=1000, out_features=64, bias=True)
  (prop1): AGNNConv()
  (prop2): AGNNConv()
  (lin2): Linear(in_features=64, out_features=3, bias=True)
)

Test Accuracy 0.4030
Net(
  (lin1): Linear(in_features=1000, out_features=64, bias=True)
  (prop1): AGNNConv()
  (prop2): AGNNConv()
  (lin2): Linear(in_features=64, out_features=3, bias=True)
)

Test Accuracy 0.4500
Net(
  (lin1): Linear(in_features=1000, out_features=64, bias=True)
  (prop1): AGNNConv()
  (prop2): AGNNConv()
  (lin2): Linear(in_features=64, out_features=3, bias=True)
)

Test Accuracy 0.4080
Net(
  (lin1): Linear(in_features=1000, out_features=64, bias=True)
  (prop1): AGNNConv()
  (prop2): AGNNConv()
  (lin2): Linear(in_features=64, out_features=3, bias=True)
)

Test Accuracy 0.4850
Net(
  (lin1): Linear(in_features=1000, out_features=64, bias=True)
  (prop1): AGNNConv()
  (prop2): AGNNConv()
  (lin2): Linear(in_features=64, out_features=3, bias=True)
)

Test Accuracy 0.4420
Net(
  (lin1): Linea

In [0]:
np.round(agnns,3).tolist()

[0.403, 0.45, 0.408, 0.485, 0.442, 0.434, 0.438, 0.457, 0.437, 0.401]

In [0]:
np.mean(agnns)

0.43549999999999994

# GAT

In [0]:
#cora上用的好像relu！！
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GATConv(num_features, 8, heads=8, dropout=0.6)
        # On the Pubmed dataset, use heads=8 in conv2.
        self.conv2 = GATConv(
            8 * 8, num_classes, heads=1, concat=True, dropout=0.6)

    def forward(self):
        x = F.dropout(data.x, p=0.6, training=self.training)
        x = F.elu(self.conv1(data.x, data.edge_index)) 
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, data.edge_index) 
        return F.log_softmax(x, dim=1)

In [0]:
# pubmed
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GATConv(num_features, 8, heads=8, dropout=0.6)
        # On the Pubmed dataset, use heads=8 in conv2.
        self.conv2 = GATConv(
            8*8, num_classes, heads=8, concat=True, dropout=0.6)

    def forward(self):
        x = F.dropout(data.x, p=0.6, training=self.training)
        x = F.elu(self.conv1(data.x, data.edge_index)) 
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, data.edge_index) 
        return F.log_softmax(x, dim=1)

In [0]:
data = r_pubmed
num_features = data.num_features
num_classes = len(set(data.y.numpy()))

In [0]:
gats = []
for _ in range(10):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model, data = Net().to(device), data.to(device)

  #r_cora_gat
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.02, weight_decay=5e-4)
  
  #pubmed_gat,都还可以
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)
  optimizer = torch.optim.Adam(model.parameters(), lr=0.02, weight_decay=5e-3)
  

  #cite_seer_gat
  #optimizer = torch.optim.Adam(model.parameters(), lr=0.02, weight_decay=5e-4)
  
  def train():
      model.train()
      optimizer.zero_grad()
      loss = F.nll_loss(model()[data.train_mask], data.y[data.train_mask])
      loss.backward()
      optimizer.step()
      return loss

  def test(mask):
      model.eval()
      with torch.no_grad():
          logits = model()
          pred = logits[mask].max(1)[1]
          acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
          
      return acc


  early_stop = True
  if early_stop:
      stopper = EarlyStopping(patience=100)
  dur = []
  print(model)

  for epoch in range(1, 101):
      model.train()
      if epoch >= 3:
          t0 = time.time()
      loss = train()
      
      if epoch >= 3:
          dur.append(time.time() - t0)
      
      val_acc = test(data.val_mask)
      
      if early_stop:
          if stopper.step(val_acc, model):   
              break
      
      '''print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | "
                " ValAcc {:.4f}".
                format(epoch, np.mean(dur), loss, val_acc))'''
      
  print() # 空出一行
      
  if early_stop:
      model.load_state_dict(torch.load('es_checkpoint.pt'))
  test_acc = test(data.test_mask)
  gats.append(test_acc)
  print("Test Accuracy {:.4f}".format(test_acc))

Net(
  (conv1): GATConv(1000, 8, heads=8)
  (conv2): GATConv(64, 3, heads=8)
)

Test Accuracy 0.4690
Net(
  (conv1): GATConv(1000, 8, heads=8)
  (conv2): GATConv(64, 3, heads=8)
)

Test Accuracy 0.4460
Net(
  (conv1): GATConv(1000, 8, heads=8)
  (conv2): GATConv(64, 3, heads=8)
)

Test Accuracy 0.4280
Net(
  (conv1): GATConv(1000, 8, heads=8)
  (conv2): GATConv(64, 3, heads=8)
)

Test Accuracy 0.4730
Net(
  (conv1): GATConv(1000, 8, heads=8)
  (conv2): GATConv(64, 3, heads=8)
)

Test Accuracy 0.4410
Net(
  (conv1): GATConv(1000, 8, heads=8)
  (conv2): GATConv(64, 3, heads=8)
)

Test Accuracy 0.4780
Net(
  (conv1): GATConv(1000, 8, heads=8)
  (conv2): GATConv(64, 3, heads=8)
)

Test Accuracy 0.4130
Net(
  (conv1): GATConv(1000, 8, heads=8)
  (conv2): GATConv(64, 3, heads=8)
)

Test Accuracy 0.4480
Net(
  (conv1): GATConv(1000, 8, heads=8)
  (conv2): GATConv(64, 3, heads=8)
)

Test Accuracy 0.4570
Net(
  (conv1): GATConv(1000, 8, heads=8)
  (conv2): GATConv(64, 3, heads=8)
)

Test Accura

In [0]:
lr=0.01, weight_decay=0.001,0.4350. 
 lr=0.02, weight_decay=5e-3, 0.4410

In [0]:
np.round(gats,3).tolist()

[0.469, 0.446, 0.428, 0.473, 0.441, 0.478, 0.413, 0.448, 0.457, 0.436]

In [0]:
np.mean(gats)

0.44889999999999997

In [0]:
0.2-5e-4: 0.4450

In [0]:
np.mean([0.427, 0.398, 0.397, 0.403, 0.376, 0.382, 0.413, 0.389, 0.371, 0.391])

0.3947